In [1]:
import requests
from pprint import pprint
import pandas as pd

In [5]:
def find_restaurants():
    
    #find location
    loc_input=input('Input your location: ').replace(' ','%20')
    url_loc = "https://developers.zomato.com/api/v2.1/locations?query={}&city_id=61".format(loc_input)
    header_loc = {"User-agent": "curl/7.43.0", "Accept": "application/json", "user_key": "{enter your key here}"}
    loc_response = (requests.get(url_loc, headers=header_loc)).json()
    
    entity_type=(loc_response['location_suggestions'][0]['entity_type'])
    entity_id=(loc_response['location_suggestions'][0]['entity_id'])
    
    #find resturants
    category=(input('Breakfast, lunch or dinner?: ')).lower()
    if category == 'breakfast':
        cat = 8
    elif category == 'lunch':
        cat = 9
    elif category == 'dinner':
        cat = 10
    else:
        print('Try again')
    count=10
    url_res = "https://developers.zomato.com/api/v2.1/search?entity_id={}&entity_type={}&category={}count={}&sort=rating&order=desc".format(entity_id,entity_type,cat,count)
    header_res = {"User-agent": "curl/7.43.0", "Accept": "application/json", "user_key": "{enter your key here}"}
    res_response = (requests.get(url_res, headers=header_res)).json()
    
    names=[res_response['restaurants'][i]['restaurant']['name'] for i in range(count)] #list of restuarant names
    postcode=[res_response['restaurants'][i]['restaurant']['location']['zipcode'] for i in range(count)]
    address=[res_response['restaurants'][i]['restaurant']['location']['address'] for i in range(count)]
    timings=[res_response['restaurants'][i]['restaurant']['timings'] for i in range(count)]
    cuisines=[res_response['restaurants'][i]['restaurant']['cuisines'] for i in range(count)]
    avg_cost=[res_response['restaurants'][i]['restaurant']['average_cost_for_two'] for i in range(count)]
    avg_rating=[res_response['restaurants'][i]['restaurant']['user_rating']['aggregate_rating'] for i in range(count)]
                
    #find hygiene rating
    format_names=[i.replace(' ','%20') for i in names]
    format_code=[i.replace(' ','%20') for i in postcode]
    hyg_rat=[]
    for i in range(len(names)):
        url_hyg = "https://api.ratings.food.gov.uk/establishments?name={}&address={}".format(format_names[i],format_code[i])
        header_hyg = {"User-agent": "curl/7.43.0", "Accept": "application/json",'x-api-version': '2'}
        hyg_response=(requests.get(url_hyg, headers=header_hyg)).json() 
        if not hyg_response['establishments']:
            hyg_rat.append('N/A')
        else:
            hyg_rat.append((hyg_response['establishments'][0]['RatingValue']))

        df = pd.DataFrame(list(zip(names,cuisines,address,timings,avg_cost,avg_rating,hyg_rat)), 
               columns =['Restaurant','cuisines','address','opening times','cost for two (£)','average rating','hygiene rating'])
    return df

find_restaurants()

Input your location: covent garden
Breakfast, lunch or dinner?: dinner


,Restaurant,cuisines,address,opening times,cost for two (£),average rating,hygiene rating
0,Dishoom,"Indian, North Indian, Curry, Cafe","12 Upper St Martin's Lane, Covent Garden, Lond...","8 AM to 11 PM (Mon-Thu),8 AM to 12 Midnight (F...",35,4.9,5
1,Chotto Matte,"Japanese, Peruvian, Sushi","11-13 Frith Street, Soho, London W1D 4RB","12 Noon to 11:30 PM (Mon-Wed, Sun), 12 Noon to...",70,4.9,4
2,Hoppers,"Sri Lankan, Indian","49 Frith Street, Soho, London W1D 4SG","12 Noon to 2:30 PM, 5:30 PM to 10:30 PM (Mon-T...",45,4.9,5
3,Amorino,"Desserts, Ice Cream, Cafe","41 Old Compton Street, Soho, London W1D 6HF","11 AM to 12 Midnight (Mon-Thu, Sun), 11 AM to ...",20,4.9,2
4,Sticks 'n' Sushi,"Japanese, Sushi","11 Henrietta Street, Covent Garden, London WC2...","12 Noon to 11 PM (Mon-Sat),12 Noon to 10 PM (Sun)",65,4.8,N/A
5,Homeslice,Pizza,"13 Neal's Yard, Covent Garden, London WC2H 9DP","12 Noon to 11 PM (Mon-Sat),12 Noon to 10 PM (Sun)",30,4.8,5
6,Clos Maggiore,"French, Seafood","33 King Street, Covent Garden, London WC2E 8JD","12 Noon to 2:30 PM, 5 PM to 11 PM (Mon-Sat),12...",80,4.8,5
7,Burger & Lobster,"American, Seafood, Burger","36 Dean Street, Soho, London W1D 4PS","12 Noon to 10:30 PM (Mon-Wed),12 Noon to 11 PM...",45,4.8,N/A
8,The Palomar,"Middle Eastern, Arabian, Moroccan","34 Rupert Street, Chinatown, London W1D 6DN","12 Noon to 2:30 PM, 5:30 PM to 11 PM (Mon-Sat)...",55,4.8,5
9,Gordon's Wine Bar,British,"47 Villiers Street, Strand, London WC2N 6NE","11 AM to 11 PM (Mon-Sat),12 Noon to 10 PM (Sun)",50,4.8,4
